# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [1]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_built():
    device = 'mps'
else:
    device = 'cpu' 

print("Device: ", device)

Device:  mps


uniform dist px goes from 0 to 1
px=0.25, exponential 2.5?
no, 0.125

In [2]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

## Data Exploration and Testing

In [3]:
datapath = '/Users/josephbajor/Dev/Datasets/11-785-f22-hw1p2/dev-clean'

idx = 83

mfcc_list = os.listdir(f'{datapath}/mfcc')
trans_list = os.listdir(f'{datapath}/transcript')

mfcc = np.load(f'{datapath}/mfcc/{mfcc_list[idx]}')
trans = np.load(f'{datapath}/transcript/{mfcc_list[idx]}')[1:-1]

In [4]:
alltrans = np.concatenate([np.load(f'{datapath}/transcript/{i}') for i in trans_list])

In [5]:
unq = np.unique(alltrans, return_counts=True)

#sil count
sil_count = unq[1][np.where(unq[0]=='SIL')[0]]
print(f'SIL count: {sil_count}')

#least common phoneme
least_common = unq[0][np.argmin(unq[1])]
print(f'Least Common: {least_common}')


SIL count: [396367]
Least Common: ZH


In [6]:
unq[1][np.where(unq[0]=='SIL')[0]]

array([396367])

In [7]:
np.unique(PHONEMES, return_inverse=True)

(array(['<eos>', '<sos>', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH',
        'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K',
        'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'SIL', 'T',
        'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH'], dtype='<U5'),
 array([32,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35,
        36, 37, 38, 39, 40, 41,  1,  0]))

In [8]:
#Mapping tests
phone_map = {lb:idx for idx, lb in enumerate(PHONEMES)}

for idx, i in enumerate(trans):
    trans[idx] = phone_map[i]


In [9]:
trans.astype(int)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, 37, 37, 37, 37, 37, 37, 37, 37,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4, 28, 28, 28, 28, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,  3,  3,  3,
        3,  3,  3,  3,  3,  3, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38,
       38,  3,  3,  3,  3,  3,  3, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       10, 10, 10,  3,  3,  3,  3,  3, 36, 36, 36, 36, 36, 36, 36, 36, 36,
       36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21,  9,  9,  9,  9, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20,  1,  1,  1, 23, 23, 23, 23, 23, 23, 23, 23, 23, 35, 35,
       35, 35, 25, 25, 25

## End of Testing Section

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

Note: Dataset functions have been modularized in another file (dataset.py)

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [10]:
config = {
    'epochs': 3,
    'batch_size' : 2048,
    'context' : 25,
    'learning_rate' : 0.001,
    'architecture' : 'very-low-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [11]:
from dataset import AudioDataset, AudioTestDataset

In [12]:
train_data = AudioDataset('/Users/josephbajor/Dev/Datasets/11-785-f22-hw1p2/train-clean-100', context=config['context']) #TODO: Create a dataset object using the AudioDataset class for the training data 

val_data = AudioDataset('/Users/josephbajor/Dev/Datasets/11-785-f22-hw1p2/dev-clean', context=config['context']) # TODO: Create a dataset object using the AudioDataset class for the validation data 

test_data = AudioTestDataset('/Users/josephbajor/Dev/Datasets/11-785-f22-hw1p2/test-clean', context=config['context']) # TODO: Create a dataset object using the AudioTestDataset class for the test data 

In [13]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 1,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 1,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 1, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  2048
Context:  25
Input size:  765
Output symbols:  42
Train dataset samples = 36191134, batches = 17672
Validation dataset samples = 1937496, batches = 947
Test dataset samples = 1943303, batches = 949


In [14]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 765]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [15]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case? Because we flattened the entire MFCC
        output_size = 40 #Why? One hot encoded

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, output_size)
        )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [16]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context']).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                 Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                        
0_model.Linear_0    [765, 40]   [2048, 40]   30640      30600
-------------------------------------------------------------
                      Totals
Total params           30640
Trainable params       30640
Non-trainable params       0
Mult-Adds              30600


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[765, 40]","[2048, 40]",30640,30600


In [17]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

30640

In [17]:
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [18]:
if device == 'cuda':
    torch.cuda.empty_cache()
    
gc.collect()

0

In [19]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    for iter, (mfccs, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
  
    train_loss /= len(dataloader)
    return train_loss

In [20]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        if device == 'cuda':
            torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = accuracy_score(phone_pred_list, phone_true_list) 
    return accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [21]:
wandb.login(key="c319fb8dfa7ce22e07aa0cefe0823a9752d50720") #API Key is in your wandb account, under settings (wandb.ai/settings)

### Create your wandb run
run = wandb.init(
    name = "test2", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="IDL-hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

wandb: Currently logged in as: jbajor. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/josephbajor/.netrc


['/Users/josephbajor/Library/CloudStorage/OneDrive-Personal/Code/CMU/Deep Learning/HW1/wandb/run-20220913_220400-32g0c24g/files/model_arch.txt']

# Experiment

In [22]:
len(train_data)

36191134

Now, it is time to finally run your ablations! Have fun!

In [23]:
# Iterate over number of epochs to train and evaluate your model
if device == 'cuda':
  torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss = train(model, optimizer, criterion, train_loader)
    accuracy = eval(model, val_loader)

    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy})

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy},
        './model_checkpoint.pth')
      
      ### Save checkpoint in wandb
      wandb.save('checkpoint.pth')

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run
run.finish()


Epoch 1/3
	Train Loss: 1.6417
	Validation Accuracy: 52.31%

Epoch 2/3
	Train Loss: 1.5751
	Validation Accuracy: 52.41%

Epoch 3/3
	Train Loss: 1.5711
	Validation Accuracy: 52.34%


train loss,█▁▁
validation accuracy,▁█▃
train loss,1.5711
validation accuracy,52.34246


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [24]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.____

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch._____:

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)

          ### Get most likely predicted phoneme with argmax
          predicted_phonemes = NotImplemented

          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
          
  return test_predictions

In [25]:
predictions = test(model, test_loader)

AttributeError: 'Network' object has no attribute '____'

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"